후보군의 노래를 가져와서 

    신나게 요리해본다


another추천
   

In [1]:
import pandas as pd
import numpy as np
import numpy.linalg
from tqdm import tqdm
from collections import Counter
import scipy.sparse as spr
import json

import datetime
import pickle

import warnings

In [2]:
song_meta = pd.read_json('data/song_meta.json')
train = pd.read_json('data/train.json')
val = pd.read_json('data/val.json')
test = pd.read_json('data/test.json')

In [3]:
ply_tagtitle = pd.read_pickle('data/transfromation/tag_title_df.pickle')

In [4]:
def get_song_date_dict( song_meta ): # song_meta를 받아서 날짜와 매핑한다.
    # 간혹 20050200 같은 놈들은 20050101
    # 0 인놈들은 1990 으로 변경
    a = []
    for t in song_meta.issue_date:
        try:
            a.append(datetime.datetime.strptime( str(t) , '%Y%m%d' ).date())
        except:
            try:
                a.append(datetime.datetime.strptime( str(t)[:4] , '%Y' ).date())
            except:
                a.append(datetime.datetime.strptime( '1990' , '%Y' ).date())
    return dict(zip(song_meta.id , a))

song_date = get_song_date_dict(song_meta) # 

def get_candidate( sim_dict , by = 'set' ):
    
    """ 후보군 뽑기
    # val_songs , val 
    # val_tags , ply_tagtitle , val
    # sim --> train + ply_tagtitle
    # by --> set , count
    """
    global song_date
    global train
    global val
    global ply_tagtitle
    val_songs = val.songs.values
    val_tags = val.tags.values
#     val_date = pd.to_datetime(val.updt_date).dt.date
    
    sim_ply = pd.merge( train[['id' , 'songs']] , ply_tagtitle , on = 'id' , how = 'left')
    res =  {}
    i = 0
    for k in tqdm(sim_dict.keys()):
        sim = sim_ply.loc[ sim_ply.id.isin( sim_dict[k] ) ]
        if by == 'set':
            cand_songs = list(set([ s for sgs in sim.songs for s in sgs if not(s in val_songs[i]) ]))  
        elif by == 'count':
            cand_songs = [ s for sgs in sim.songs for s in sgs if not(s in val_songs[i]) ]  
# and (song_date[s] < val_date[i])]
        cand_tags = [ t for tgs in sim.complex_col for t in tgs if not (t in val_tags[i]) ]
        res[k] = { 'songs' : cand_songs  , 'tags' : cand_tags }
        i += 1
    return res

def filter_date( candi ):
    global val
    val_date = pd.to_datetime(val.updt_date).dt.date
    j = 0 
    for i in tqdm(candi):
        candi[i]['songs'] = [ s for s in candi[i]['songs'] if song_date[s] <= val_date[j]]
        j += 1
    return candi

def load_pickle( f_path ):
    with open( f_path , 'rb') as f:
        return pickle.load(f)
    

<h3></h3>
<h3></h3>

**연관있는 플레이리스트를 담은 pickle 불러오기**

In [11]:
s_sim = load_pickle( 'candi/res_ver34_song.pickle' )
# w_sim = load_pickle( 'candi/res_ver34_word.pickle' )
# sin_sim = load_pickle( 'candi/res_ver34_singer.pickle' )
t_sim = load_pickle( 'candi/res_ver34_tag.pickle' )
# g_sim = load_pickle( 'candi/res_ver34_gnr.pickle' )
# ply_sim = load_pickle( 'candi/res_ver34_ply.pickle' )
# alb_sim = load_pickle( 'candi/res_ver34_album.pickle')

<h3></h3>
<h3></h3>


### 후보군 중에서 추천 해주기 

- 어떠한 방법으로 할까??

1. 집합의 개념으로? 각영역이 추천한 노래들의 비중을 모두 1로 환산

2. 단순 집계 모두 더한다음에 가장 많은 노래를 뱉기

1. 집합 방식의 추첨

<h3></h3>
<h3></h3>


**필터링별 데이터 받기**

In [6]:
s_can = get_candidate( s_sim , by = 'count'  )
s_can = filter_date( s_can )

100%|██████████| 23015/23015 [01:06<00:00, 343.54it/s]


In [8]:
w_can = get_candidate( w_sim , by = 'count')
w_can = filter_date( w_can )

100%|██████████| 23015/23015 [00:59<00:00, 383.79it/s]


In [10]:
sin_can = get_candidate( sin_sim, by = 'count' )
sin_can = filter_date( sin_can )

100%|██████████| 23015/23015 [01:01<00:00, 372.14it/s]


In [12]:
t_can = get_candidate( t_sim, by = 'count' )
t_can = filter_date( t_can )

100%|██████████| 23015/23015 [01:12<00:00, 316.90it/s]


In [13]:
g_sim = load_pickle( 'candi/res_ver34_gnr.pickle' )
g_can = get_candidate( g_sim, by = 'count' )
g_can = filter_date( g_can )

100%|██████████| 23015/23015 [01:17<00:00, 297.51it/s]


In [15]:
ply_sim = load_pickle( 'candi/res_ver34_ply.pickle' )
ply_can = get_candidate( ply_sim, by = 'count' )
ply_can = filter_date( ply_can )

100%|██████████| 23015/23015 [01:03<00:00, 363.03it/s]


In [14]:
alb_sim = load_pickle( 'candi/res_ver34_album.pickle')
alb_can = get_candidate( alb_sim, by = 'count' )
alb_can = filter_date( alb_can )

100%|██████████| 23015/23015 [05:01<00:00, 76.28it/s] 


In [29]:
SAG_sim = load_pickle( 'candi/res_ver37_SAG.pickle')
SAG_can = get_candidate( SAG_sim, by = 'count' )
SAG_can = filter_date( SAG_can )

100%|██████████| 23015/23015 [01:20<00:00, 285.26it/s]


In [31]:
SWG_sim = load_pickle( 'candi/res_ver37_SWG.pickle')
SWG_can = get_candidate( SWG_sim, by = 'count' )
SWG_can = filter_date( SWG_can )

100%|██████████| 23015/23015 [00:56<00:00, 406.44it/s]


In [39]:
SWAG_sim = load_pickle('candi/res_ver39_SWAG.pickle')
SWAG_can = get_candidate( SWAG_sim, by = 'count' )
SWAG_can = filter_date( SWAG_can )

100%|██████████| 23015/23015 [01:00<00:00, 378.47it/s]


In [48]:
TSA_sim = load_pickle('candi/res_ver40_TSA.pickle')
TSA_can = get_candidate( TSA_sim, by = 'count' )
TSA_can = filter_date( TSA_can )

100%|██████████| 23015/23015 [01:17<00:00, 296.85it/s]


In [50]:
WAT_sim = load_pickle('candi/res_ver40_WAT.pickle')
WAT_can = get_candidate( WAT_sim, by = 'count' )
WAT_can = filter_date( WAT_can )

100%|██████████| 23015/23015 [01:03<00:00, 361.34it/s]


In [51]:
SWAT_sim = load_pickle('candi/res_ver40_SWAT.pickle')
SWAT_can = get_candidate( SWAT_sim, by = 'count' )
SWAT_can = filter_date( SWAT_can )

100%|██████████| 23015/23015 [00:52<00:00, 441.61it/s]


In [52]:
candi = {}
for i in tqdm( ply_can.keys() ):
    candi[i] = { 'songs' : s_can[i]['songs'] + w_can[i]['songs'] + sin_can[i]['songs'] + t_can[i]['songs'] + g_can.get(i)['songs'] + ply_can[i]['songs']+alb_can[i]['songs']+SAG_can[i]['songs']+SWG_can[i]['songs']+SWAG_can[i]['songs']+TSA_can[i]['songs']+WAT_can[i]['songs']+SWAT_can[i]['songs'] ,
                   'tags': s_can[i]['tags'] + w_can[i]['tags'] + sin_can[i]['tags'] + t_can[i]['tags'] + g_can[i]['tags'] + ply_can[i]['tags'] + alb_can[i]['tags'] + SAG_can[i]['tags'] + SWG_can[i]['tags']+SWAG_can[i]['tags']+TSA_can[i]['tags']+WAT_can[i]['tags']+SWAT_can[i]['tags']}

100%|██████████| 23015/23015 [00:11<00:00, 2026.60it/s]


In [ ]:
candi = {}
for i in tqdm( ply_can.keys() ):
    candi[i] = { 'songs' : s_can[i]['songs'] + w_can[i]['songs'] + sin_can[i]['songs'] + t_can[i]['songs'] + g_can.get(i)['songs'] + ply_can[i]['songs'] ,
                   'tags': s_can[i]['tags'] + w_can[i]['tags'] + sin_can[i]['tags'] + t_can[i]['tags'] + g_can[i]['tags'] + ply_can[i]['tags'] }

    

**후보군이 적당히 뽑혔는지**

In [53]:
a=0
b=0
for i in candi:
    if len(set(candi[i]['songs'])) < 100:
        a += 1
    if len(set(candi[i]['tags'])) < 10:
        b += 1
        
print( f'노래 하자 있는 ply{a}')
print( f'태그 하자 있는 ply{b}')

노래 하자 있는 ply1
태그 하자 있는 ply0


In [54]:
ply = {}
for i in tqdm(candi):
    ply[i] = {'songs' : [ l[0] for l in Counter(candi[i]['songs']).most_common(100) ],
                'tags' : [l[0] for l in Counter(candi[i]['tags']).most_common(10) ]} 
    

100%|██████████| 23015/23015 [00:18<00:00, 1238.65it/s]


In [55]:
print( '**인성 문제있는 플레이리스트**')
haja_ply_song = []
for i in ply.keys():
    if len(ply[i]['songs']) != 100:
        a = len(set(ply[i]['songs']))
        haja_ply_song.append(i)
        print( f'{i} : {a}' )
        
            
haja_ply_tag = []            
print( '**인성 문제있는 태그**' )
for i in ply.keys():
    if len(ply[i]['tags']) != 10:
        a = len(ply[i]['tags'])
        haja_ply_tag.append(i)
        print( f'{i} : {a}' )

**인성 문제있는 플레이리스트**
69439 : 94
**인성 문제있는 태그**


### 응급 처치

In [23]:
with open( 'data/model/model_3_another.pickle' , 'rb') as f:
    faid = pickle.load(f)

In [24]:
song_meta = pd.read_json( 'data/song_meta.json' )

In [56]:

for i in tqdm(haja_ply_song):
    top_gnr = Counter([g for gs in song_meta.loc[ song_meta.id.isin( ply[i]['songs'] ) ].song_gn_gnr_basket for g in gs]).most_common(1)[0][0]
    tmp = faid[top_gnr]
    for s in ply[i]['songs']:
        try:
            tmp.remove(s)
        except:
            pass
    
    ply[i]['songs'] += tmp[: 100 - len(ply[i]['songs'])]

100%|██████████| 1/1 [00:00<00:00, 30.09it/s]


In [58]:
print( '**인성 문제있는 플레이리스트**')
haja_ply_song = []
for i in ply.keys():
    if len(ply[i]['songs']) != len(set(ply[i]['songs'])):
        a = len(set(ply[i]['songs']))
        haja_ply_song.append(i)
        print( f'{i} : {a}' )
       
    
haja_ply_tag = []            
print( '**인성 문제있는 태그**' )
for i in ply.keys():
    if len(ply[i]['tags']) != len(set(ply[i]['tags'])):
        a = len(ply[i]['tags'])
        haja_ply_tag.append(i)
        print( f'{i} : {a}' )

**인성 문제있는 플레이리스트**
**인성 문제있는 태그**


In [60]:
res = [] 
a = 0

res = [{ 'id' : i , 'songs' : ply[i]['songs'] , 'tags' : ply[i]['tags'] } for i in ply]

In [61]:
with open( 'result/res_ver40.json' , 'w' ) as f:
    json.dump( res , f )